# Fine Tuning Roberta for Sentiment Analysis





In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 8.3MB/s 
     |████████████████████████████████| 1.2MB 38.0MB/s 
     |████████████████████████████████| 3.0MB 41.3MB/s 
     |████████████████████████████████| 901kB 36.5MB/s 


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g6CVIeMTuX-XTqC2oDv-1OzJ8fL8TjXtdX5nuq7gASkD5gWq1ZwW_c
Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/training.1600000.processed.noemoticon_simple_proc.csv',encoding='latin-1', header=None)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df = df.dropna(axis='rows')

X = df[7][1:]
y = df[8][1:].apply(lambda t: int(t))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666, 
                                                        shuffle=True, stratify=y)
df_train = pd.DataFrame({'txt': X_train, 'target':y_train})
df_test = pd.DataFrame({'txt': X_test, 'target':y_test})
df_train['target'] = df_train['target'].apply(lambda t: 0 if t == 0 else 1)
df_test['target'] = df_test['target'].apply(lambda t: 0 if t == 0 else 1)
df_train = df_train.iloc[:100000]
df_test = df_test.iloc[:20000]


In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
# TRAIN_BATCH_SIZE = 8

# VALID_BATCH_SIZE = 4
VALID_BATCH_SIZE = 32
# EPOCHS = 1
LEARNING_RATE = 3e-05
# LEARNING_RATE = 1e-05

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.txt
        self.targets = self.data.target
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(df_train.shape))
print("TEST Dataset: {}".format(df_test.shape))

training_set = SentimentData(df_train, tokenizer, MAX_LEN)
testing_set = SentimentData(df_test, tokenizer, MAX_LEN)

FULL Dataset: (1562118, 9)
TRAIN Dataset: (100000, 2)
TEST Dataset: (20000, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        # self.classifier = torch.nn.Linear(768, 5)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = RobertaClass()
model.to(device)

In [ ]:
model = torch.load('/content/drive/MyDrive/Colab Notebooks/pytorch_roberta_part_sentiment.bin')
model.eval()

RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, eleme

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        optimizer.zero_grad()
        
        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%500==0:
            output_model_file = f'/content/drive/MyDrive/Colab Notebooks/pytorch_roberta_part_sentiment.bin'
            output_vocab_file = '/content/drive/MyDrive/Colab Notebooks/'

            torch.save(model, output_model_file)
            tokenizer.save_vocabulary(output_vocab_file)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        # optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
EPOCHS = 3
for epoch in range(EPOCHS):
    train(epoch)











0it [00:00, ?it/s]









1it [00:00,  6.18it/s]

Training Loss per 5000 steps: 1.6329421997070312
Training Accuracy per 5000 steps: 12.5


Выходные данные были обрезаны до нескольких последних строк (5000).









4502it [10:58,  6.90it/s]









4503it [10:59,  6.91it/s]









4504it [10:59,  6.94it/s]









4505it [10:59,  6.97it/s]









4506it [10:59,  7.00it/s]









4507it [10:59,  6.91it/s]









4508it [10:59,  6.96it/s]









4509it [10:59,  7.01it/s]









4510it [11:00,  6.95it/s]









4511it [11:00,  6.87it/s]









4512it [11:00,  6.90it/s]









4513it [11:00,  6.94it/s]









4514it [11:00,  6.85it/s]









4515it [11:00,  6.90it/s]









4516it [11:00,  6.95it/s]









4517it [11:01,  6.93it/s]









4518it [11:01,  6.82it/s]









4519it [11:01,  6.79it/s]









4520it [11:01,  6.88it/s]









4521it [11:01,  6.88it/s]









4522it [11:01,  6.94it/s]









4523it [11:01,  6.86it/s]









4524it [11:02,  6.82it/s]









4525it [11:02,  6.84it/s]









4526it [11:02,  6.90it/s]









4527it [11:02,  6.82it/s]









4528it [11:02

Training Loss per 5000 steps: 0.5850325608683739
Training Accuracy per 5000 steps: 69.18366326734653


Выходные данные были обрезаны до нескольких последних строк (5000).









9502it [23:15,  6.94it/s]









9503it [23:15,  6.88it/s]









9504it [23:15,  6.93it/s]









9505it [23:15,  6.85it/s]









9506it [23:16,  6.82it/s]









9507it [23:16,  6.88it/s]









9508it [23:16,  6.85it/s]









9509it [23:16,  6.85it/s]









9510it [23:16,  6.79it/s]









9511it [23:16,  6.65it/s]









9512it [23:16,  6.60it/s]









9513it [23:17,  6.69it/s]









9514it [23:17,  6.66it/s]









9515it [23:17,  6.79it/s]









9516it [23:17,  6.88it/s]









9517it [23:17,  6.70it/s]









9518it [23:17,  6.74it/s]









9519it [23:17,  6.76it/s]









9520it [23:18,  6.80it/s]









9521it [23:18,  6.76it/s]









9522it [23:18,  6.81it/s]









9523it [23:18,  6.77it/s]









9524it [23:18,  6.70it/s]









9525it [23:18,  6.82it/s]









9526it [23:19,  6.78it/s]









9527it [23:19,  6.76it/s]









9528it [23:19

...

Training Loss per 5000 steps: 0.47116104713042384
Training Accuracy per 5000 steps: 77.51949610077985


Выходные данные были обрезаны до нескольких последних строк (5000).









9502it [22:45,  7.12it/s]









9503it [22:45,  7.12it/s]









9504it [22:45,  7.09it/s]









9505it [22:45,  6.99it/s]









9506it [22:46,  6.93it/s]









9507it [22:46,  6.92it/s]









9508it [22:46,  6.95it/s]









9509it [22:46,  7.04it/s]









9510it [22:46,  7.08it/s]









9511it [22:46,  7.12it/s]









9512it [22:46,  7.12it/s]









9513it [22:47,  7.15it/s]









9514it [22:47,  7.13it/s]









9515it [22:47,  7.07it/s]









9516it [22:47,  6.93it/s]









9517it [22:47,  7.01it/s]









9518it [22:47,  7.01it/s]









9519it [22:47,  7.08it/s]









9520it [22:48,  7.06it/s]









9521it [22:48,  7.03it/s]









9522it [22:48,  7.00it/s]









9523it [22:48,  7.07it/s]









9524it [22:48,  7.10it/s]









9525it [22:48,  7.07it/s]









9526it [22:48,  7.12it/s]









9527it [22:49,  7.09it/s]









9528it [22:49

Training Loss per 5000 steps: 0.4745946564603515
Training Accuracy per 5000 steps: 77.2997700229977


Выходные данные были обрезаны до нескольких последних строк (5000).









12001it [28:48,  6.85it/s]









12002it [28:48,  6.82it/s]









12003it [28:49,  6.79it/s]









12004it [28:49,  6.78it/s]









12005it [28:49,  6.78it/s]









12006it [28:49,  6.67it/s]









12007it [28:49,  6.70it/s]









12008it [28:49,  6.75it/s]









12009it [28:49,  6.85it/s]









12010it [28:50,  6.92it/s]









12011it [28:50,  6.92it/s]









12012it [28:50,  6.92it/s]









12013it [28:50,  6.88it/s]









12014it [28:50,  6.78it/s]









12015it [28:50,  6.73it/s]









12016it [28:50,  6.81it/s]









12017it [28:51,  6.55it/s]









12018it [28:51,  6.57it/s]









12019it [28:51,  6.51it/s]









12020it [28:51,  6.67it/s]









12021it [28:51,  6.70it/s]









12022it [28:51,  6.80it/s]









12023it [28:51,  6.88it/s]









12024it [28:52,  6.79it/s]









12025it [28:52,  6.82it/s]









12026it [28:52,  6.78it

The Total Accuracy for Epoch 2: 77.228
Training Loss Epoch: 0.4755350073191524
Training Accuracy Epoch: 77.228


In [ ]:
# #V2
EPOCHS = 3
for epoch in range(EPOCHS):
    train(epoch)
# torch.cuda.empty_cache()

0it [00:00, ?it/s]

Training Loss per 5000 steps: 0.6808712482452393
Training Accuracy per 5000 steps: 56.25


3125it [23:03,  2.26it/s]
0it [00:00, ?it/s]

The Total Accuracy for Epoch 0: 71.195
Training Loss Epoch: 0.5572323099803924
Training Accuracy Epoch: 71.195
Training Loss per 5000 steps: 0.41330063343048096
Training Accuracy per 5000 steps: 78.125


3125it [23:00,  2.26it/s]
0it [00:00, ?it/s]

The Total Accuracy for Epoch 1: 74.827
Training Loss Epoch: 0.5125647881221771
Training Accuracy Epoch: 74.827
Training Loss per 5000 steps: 0.3899500072002411
Training Accuracy per 5000 steps: 90.625


3125it [23:02,  2.26it/s]

The Total Accuracy for Epoch 2: 77.099
Training Loss Epoch: 0.4795297661972046
Training Accuracy Epoch: 77.099


In [ ]:
#V3 
EPOCHS = 3
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]

Training Loss per 5000 steps: 0.6867580413818359
Training Accuracy per 5000 steps: 53.125


3125it [39:54,  1.30it/s]
0it [00:00, ?it/s]

The Total Accuracy for Epoch 0: 71.242
Training Loss Epoch: 0.5581089242935181
Training Accuracy Epoch: 71.242
Training Loss per 5000 steps: 0.6112483143806458
Training Accuracy per 5000 steps: 68.75


3125it [39:54,  1.31it/s]
0it [00:00, ?it/s]

The Total Accuracy for Epoch 1: 74.744
Training Loss Epoch: 0.5133457920026779
Training Accuracy Epoch: 74.744
Training Loss per 5000 steps: 0.45797988772392273
Training Accuracy per 5000 steps: 81.25


3125it [39:55,  1.30it/s]

The Total Accuracy for Epoch 2: 77.162
Training Loss Epoch: 0.4796179421901703
Training Accuracy Epoch: 77.162


In [ ]:
#v4
EPOCHS = 3
for epoch in range(EPOCHS):
    train(epoch)
    output_model_file = f'/content/drive/MyDrive/Colab Notebooks/pytorch_roberta_part_sentiment.bin'
    output_vocab_file = '/content/drive/MyDrive/Colab Notebooks/'

    torch.save(model, output_model_file)
    tokenizer.save_vocabulary(output_vocab_file)

0it [00:00, ?it/s]

Training Loss per 5000 steps: 0.6836727857589722
Training Accuracy per 5000 steps: 53.125


3125it [40:37,  1.28it/s]


The Total Accuracy for Epoch 0: 71.093
Training Loss Epoch: 0.5579593627548218
Training Accuracy Epoch: 71.093


0it [00:00, ?it/s]

...

The Total Accuracy for Epoch 1: 74.715
Training Loss Epoch: 0.5127406526088715
Training Accuracy Epoch: 74.715


0it [00:00, ?it/s]

Training Loss per 5000 steps: 0.40102505683898926
Training Accuracy per 5000 steps: 81.25


3125it [40:35,  1.28it/s]


The Total Accuracy for Epoch 2: 76.977
Training Loss Epoch: 0.4798540447473526
Training Accuracy Epoch: 76.977


In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu


In [ ]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)











0it [00:00, ?it/s]









4it [00:00, 39.94it/s]

Validation Loss per 100 steps: 0.17866311967372894
Validation Accuracy per 100 steps: 100.0


Выходные данные были обрезаны до нескольких последних строк (5000).









2518it [01:01, 39.95it/s]









2523it [01:01, 39.69it/s]









2527it [01:01, 39.04it/s]









2531it [01:01, 38.92it/s]









2536it [01:01, 39.72it/s]









2541it [01:02, 40.20it/s]









2546it [01:02, 40.00it/s]









2551it [01:02, 40.56it/s]









2556it [01:02, 41.08it/s]









2561it [01:02, 41.41it/s]









2566it [01:02, 41.70it/s]









2571it [01:02, 41.87it/s]









2576it [01:02, 41.62it/s]









2581it [01:02, 41.56it/s]









2586it [01:03, 41.63it/s]









2591it [01:03, 41.22it/s]









2596it [01:03, 40.62it/s]









2601it [01:03, 40.38it/s]









2606it [01:03, 40.29it/s]









2611it [01:03, 40.69it/s]









2616it [01:03, 40.86it/s]









2621it [01:03, 40.55it/s]









2626it [01:04, 40.81it/s]









2631it [01:04, 40.56it/s]









2636it [01:04, 41.08it/s]









2641it [01:04, 40.50it/s]









2646it [01:04

Validation Loss Epoch: 0.526577581608668
Validation Accuracy Epoch: 74.055
Accuracy on test data = 74.06%


In [ ]:
# v2
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

1it [00:00,  6.73it/s]

Validation Loss per 100 steps: 0.4792912006378174
Validation Accuracy per 100 steps: 78.125


625it [01:34,  6.63it/s]

Validation Loss Epoch: 0.5235367028713226
Validation Accuracy Epoch: 73.955
Accuracy on test data = 73.95%


In [ ]:
# v2
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

1it [00:00,  3.50it/s]

Validation Loss per 100 steps: 0.3650934100151062
Validation Accuracy per 100 steps: 78.125


625it [02:46,  3.75it/s]

Validation Loss Epoch: 0.5386569926023483
Validation Accuracy Epoch: 74.4
Accuracy on test data = 74.40%


In [ ]:
# v2
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

1it [00:00,  3.43it/s]

Validation Loss per 100 steps: 0.6606760621070862
Validation Accuracy per 100 steps: 75.0


625it [02:46,  3.76it/s]

Validation Loss Epoch: 0.5517327502250672
Validation Accuracy Epoch: 73.975
Accuracy on test data = 73.97%


In [ ]:
# v2
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

1it [00:00,  2.64it/s]

Validation Loss per 100 steps: 0.6947959065437317
Validation Accuracy per 100 steps: 65.625


625it [02:50,  3.67it/s]

Validation Loss Epoch: 0.5517327470302582
Validation Accuracy Epoch: 73.975
Accuracy on test data = 73.97%


In [ ]:
output_model_file = 'pytorch_roberta_v2_sentiment.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')

All files saved
This tutorial is completed


In [ ]:
twitter_labelled_dta = pd.read_json('/content/drive/MyDrive/Colab Notebooks/simple_proc_twitter_labelled.json')
twitter_labelled_dta_1 = twitter_labelled_dta[twitter_labelled_dta['sentiment'] != 0]
twitter_labelled_dta_1['target'] = twitter_labelled_dta_1['sentiment'].apply(lambda t: 0 if t == -1 else 1)
twitter_labelled_dta_1['txt'] = twitter_labelled_dta_1['text_simple_proc']
df_test_twitter = twitter_labelled_dta_1.reset_index(drop=True)

twitter_testing_set = SentimentData(df_test_twitter, tokenizer, MAX_LEN)

twitter_testing_loader = DataLoader(twitter_testing_set, **test_params)

acc = valid(model, twitter_testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.










0it [00:00, ?it/s]









4it [00:00, 36.24it/s]









8it [00:00, 37.06it/s]

Validation Loss per 100 steps: 0.2944850027561188
Validation Accuracy per 100 steps: 75.0












12it [00:00, 37.23it/s]









16it [00:00, 37.70it/s]









21it [00:00, 38.73it/s]









25it [00:00, 39.00it/s]









30it [00:00, 39.64it/s]









35it [00:00, 40.06it/s]









40it [00:01, 40.32it/s]









45it [00:01, 40.88it/s]









49it [00:01, 40.58it/s]









54it [00:01, 41.08it/s]









59it [00:01, 41.08it/s]









64it [00:01, 41.26it/s]









69it [00:01, 41.21it/s]









74it [00:01, 41.41it/s]









79it [00:01, 41.17it/s]









84it [00:02, 41.07it/s]









89it [00:02, 41.33it/s]









94it [00:02, 41.14it/s]









99it [00:02, 40.77it/s]









104it [00:02, 40.74it/s]









109it [00:02, 40.57it/s]









114it [00:02, 40.44it/s]









119it [00:02, 40.49it/s]









124it [00:03, 40.40it/s]









129it [00:03, 40.45it/s]









134it [00:03, 40.78it/s]









139it [00:03, 40.42it/s]









144it [00:03, 40.66it/s]









149it [00:03, 40.75it/s]









154it [00:03, 40.72it/s

Validation Loss Epoch: 0.5056766909212698
Validation Accuracy Epoch: 73.82271468144044
Accuracy on test data = 73.82%


In [ ]:
def load_accuracy_for_labelled(type):
  _df = pd.read_json(f'/content/drive/MyDrive/Colab Notebooks/simple_proc_{type}_labelled.json')
  _df = _df.dropna(axis='rows')
  _df_1 = _df[_df['sentiment'] != 0]
  _df_1['target'] = _df_1['sentiment'].apply(lambda t: 0 if t == -1 else 1)
  _df_1['txt'] = _df_1['text_simple_proc']
  _df_1 = _df_1.reset_index(drop=True)

  _testing_set = SentimentData(_df_1, tokenizer, MAX_LEN)
  _testing_loader = DataLoader(_testing_set, **test_params)

  acc = valid(model, _testing_loader)
  print(type)
  print("Accuracy on test data = %0.2f%%" % acc)

In [ ]:
load_accuracy_for_labelled('twitter')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  











0it [00:00, ?it/s]










4it [00:00, 38.98it/s]










8it [00:00, 38.96it/s]

Validation Loss per 100 steps: 0.6651346683502197
Validation Accuracy per 100 steps: 75.0













12it [00:00, 38.95it/s]










17it [00:00, 39.64it/s]










22it [00:00, 40.20it/s]










27it [00:00, 40.66it/s]










32it [00:00, 40.81it/s]










37it [00:00, 41.37it/s]










42it [00:01, 41.65it/s]










47it [00:01, 41.73it/s]










52it [00:01, 41.99it/s]










57it [00:01, 42.15it/s]










62it [00:01, 42.23it/s]










67it [00:01, 42.34it/s]










72it [00:01, 41.58it/s]










77it [00:01, 41.96it/s]










82it [00:01, 42.20it/s]










87it [00:02, 42.17it/s]










92it [00:02, 42.13it/s]










97it [00:02, 41.81it/s]










102it [00:02, 42.00it/s]










107it [00:02, 41.91it/s]










112it [00:02, 41.88it/s]










117it [00:02, 42.12it/s]










122it [00:02, 42.42it/s]










127it [00:03, 41.21it/s]










132it [00:03, 41.31it/s]










137it [00:03, 41.62it/s]










142it [00:03, 41.73it/s]










147it [00:03, 41.61it/s]










152it [00:03, 41.49it/s]




Validation Loss Epoch: 0.5026766004977306
Validation Accuracy Epoch: 73.82271468144044
twitter
Accuracy on test data = 73.82%


In [ ]:
#v2
load_accuracy_for_labelled('twitter')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
1it [00:00,  3.66it/s]

Validation Loss per 100 steps: 0.47374749183654785
Validation Accuracy per 100 steps: 75.0


23it [00:06,  3.75it/s]

Validation Loss Epoch: 0.5109427260315936
Validation Accuracy Epoch: 76.3157894736842
twitter
Accuracy on test data = 76.32%


In [ ]:
#v2
load_accuracy_for_labelled('twitter')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
1it [00:00,  3.68it/s]

Validation Loss per 100 steps: 0.6689332723617554
Validation Accuracy per 100 steps: 71.875


23it [00:06,  3.75it/s]

Validation Loss Epoch: 0.4748146352560624
Validation Accuracy Epoch: 77.28531855955679
twitter
Accuracy on test data = 77.29%


In [ ]:
#v3
load_accuracy_for_labelled('twitter')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
1it [00:00,  3.57it/s]

Validation Loss per 100 steps: 0.39857959747314453
Validation Accuracy per 100 steps: 84.375


23it [00:06,  3.66it/s]

Validation Loss Epoch: 0.47545935537504114
Validation Accuracy Epoch: 77.28531855955679
twitter
Accuracy on test data = 77.29%


In [ ]:
# load_accuracy_for_labelled('reddit')
twitter_labelled_dta = pd.read_json('/content/drive/MyDrive/Colab Notebooks/simple_proc_reddit_labelled.json')
twitter_labelled_dta_1 = twitter_labelled_dta[twitter_labelled_dta['sentiment'] != 0]
twitter_labelled_dta_1['target'] = twitter_labelled_dta_1['sentiment'].apply(lambda t: 0 if t == -1 else 1)
twitter_labelled_dta_1['txt'] = twitter_labelled_dta_1['text_simple_proc']
df_test_twitter = df_test_twitter.iloc[:1500]
df_test_twitter = twitter_labelled_dta_1.reset_index(drop=True)

twitter_testing_set = SentimentData(df_test_twitter, tokenizer, MAX_LEN)

twitter_testing_loader = DataLoader(twitter_testing_set, **test_params)

model.eval()
n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
with torch.no_grad():
  for _, data in tqdm(enumerate(twitter_testing_loader, 0)):
    ids = data['ids'].to(device, dtype = torch.long)
    mask = data['mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
    targets = data['targets'].to(device, dtype = torch.long)
    outputs = model(ids, mask, token_type_ids).squeeze()
    # print(outputs)
    # print(targets)
    try:
      loss = loss_function(outputs, targets)
      tr_loss += loss.item()
      big_val, big_idx = torch.max(outputs.data, dim=1)
      n_correct += calcuate_accuracy(big_idx, targets)

      nb_tr_steps += 1
      nb_tr_examples+=targets.size(0)
    except Exception:
      print('error')
            
    if _%5000==0:
      loss_step = tr_loss/nb_tr_steps
      accu_step = (n_correct*100)/nb_tr_examples
      print(f"Validation Loss per 100 steps: {loss_step}")
      print(f"Validation Accuracy per 100 steps: {accu_step}")
epoch_loss = tr_loss/nb_tr_steps
epoch_accu = (n_correct*100)/nb_tr_examples
print(f"Validation Loss Epoch: {epoch_loss}")
print(f"Validation Accuracy Epoch: {epoch_accu}")
    
acc = epoch_accu
print("Accuracy on test data = %0.2f%%" % acc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """















0it [00:00, ?it/s]














4it [00:00, 37.52it/s]














8it [00:00, 37.86it/s]

Validation Loss per 100 steps: 0.8237903118133545
Validation Accuracy per 100 steps: 50.0

















12it [00:00, 38.21it/s]














17it [00:00, 39.10it/s]














22it [00:00, 39.58it/s]














27it [00:00, 40.18it/s]














32it [00:00, 40.79it/s]














37it [00:00, 40.94it/s]














42it [00:01, 41.03it/s]














46it [00:01, 40.52it/s]














51it [00:01, 41.11it/s]














56it [00:01, 41.46it/s]














61it [00:01, 41.69it/s]














66it [00:01, 41.93it/s]














71it [00:01, 41.57it/s]














76it [00:01, 41.81it/s]














81it [00:01, 41.89it/s]














86it [00:02, 42.19it/s]














91it [00:02, 42.28it/s]














96it [00:02, 42.29it/s]














101it [00:02, 41.46it/s]














106it [00:02, 41.45it/s]














111it [00:02, 41.24it/s]














116it [00:02, 41.48it/s]














121it [00:02, 41.42it/s]














126it [00:03, 41.52it/s]














131it [00:03, 41.38it/s]














136it [00:03, 41.23it/s]





error
Validation Loss Epoch: 0.5410765929763287
Validation Accuracy Epoch: 70.18828451882845
Accuracy on test data = 70.19%


In [ ]:
#v2
# load_accuracy_for_labelled('reddit')
twitter_labelled_dta = pd.read_json('/content/drive/MyDrive/Colab Notebooks/simple_proc_reddit_labelled.json')
twitter_labelled_dta_1 = twitter_labelled_dta[twitter_labelled_dta['sentiment'] != 0]
twitter_labelled_dta_1['target'] = twitter_labelled_dta_1['sentiment'].apply(lambda t: 0 if t == -1 else 1)
twitter_labelled_dta_1['txt'] = twitter_labelled_dta_1['text_simple_proc']
# df_test_twitter = df_test_twitter.iloc[:1500]
df_test_twitter = twitter_labelled_dta_1.reset_index(drop=True)

twitter_testing_set = SentimentData(df_test_twitter, tokenizer, MAX_LEN)

twitter_testing_loader = DataLoader(twitter_testing_set, **test_params)

model.eval()
n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
with torch.no_grad():
  for _, data in tqdm(enumerate(twitter_testing_loader, 0)):
    ids = data['ids'].to(device, dtype = torch.long)
    mask = data['mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
    targets = data['targets'].to(device, dtype = torch.long)
    outputs = model(ids, mask, token_type_ids).squeeze()
    # print(outputs)
    # print(targets)
    try:
      loss = loss_function(outputs, targets)
      tr_loss += loss.item()
      big_val, big_idx = torch.max(outputs.data, dim=1)
      n_correct += calcuate_accuracy(big_idx, targets)

      nb_tr_steps += 1
      nb_tr_examples+=targets.size(0)
    except Exception:
      print('error')
            
    if _%5000==0:
      loss_step = tr_loss/nb_tr_steps
      accu_step = (n_correct*100)/nb_tr_examples
      print(f"Validation Loss per 100 steps: {loss_step}")
      print(f"Validation Accuracy per 100 steps: {accu_step}")
epoch_loss = tr_loss/nb_tr_steps
epoch_accu = (n_correct*100)/nb_tr_examples
print(f"Validation Loss Epoch: {epoch_loss}")
print(f"Validation Accuracy Epoch: {epoch_accu}")
    
acc = epoch_accu
print("Accuracy on test data = %0.2f%%" % acc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
1it [00:00,  3.58it/s]

Validation Loss per 100 steps: 0.42606011033058167
Validation Accuracy per 100 steps: 81.25


30it [00:08,  3.70it/s]

Validation Loss Epoch: 0.5143252809842428
Validation Accuracy Epoch: 74.8171368861024
Accuracy on test data = 74.82%


In [ ]:
load_accuracy_for_labelled('reddit')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
1it [00:00,  3.69it/s]

Validation Loss per 100 steps: 0.514586329460144
Validation Accuracy per 100 steps: 78.125


30it [00:08,  3.71it/s]

Validation Loss Epoch: 0.5077965945005417
Validation Accuracy Epoch: 74.71264367816092
reddit
Accuracy on test data = 74.71%


In [ ]:
load_accuracy_for_labelled('reddit') #v4

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
1it [00:00,  3.52it/s]

Validation Loss per 100 steps: 0.7701587677001953
Validation Accuracy per 100 steps: 59.375


30it [00:08,  3.61it/s]

Validation Loss Epoch: 0.5072363366683325
Validation Accuracy Epoch: 74.71264367816092
reddit
Accuracy on test data = 74.71%


In [ ]:
load_accuracy_for_labelled('instagram')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  












0it [00:00, ?it/s]











4it [00:00, 38.24it/s]











8it [00:00, 38.14it/s]

Validation Loss per 100 steps: 0.24626661837100983
Validation Accuracy per 100 steps: 100.0














12it [00:00, 38.19it/s]











17it [00:00, 38.82it/s]











22it [00:00, 39.50it/s]











26it [00:00, 38.97it/s]











31it [00:00, 39.73it/s]











35it [00:00, 39.63it/s]











39it [00:00, 39.58it/s]











44it [00:01, 39.77it/s]











48it [00:01, 39.50it/s]











52it [00:01, 39.38it/s]











57it [00:01, 39.93it/s]











61it [00:01, 39.79it/s]











65it [00:01, 39.63it/s]











69it [00:01, 39.40it/s]











74it [00:01, 39.84it/s]











79it [00:01, 40.52it/s]











84it [00:02, 40.71it/s]











89it [00:02, 40.41it/s]











94it [00:02, 40.14it/s]











99it [00:02, 40.11it/s]











104it [00:02, 40.23it/s]











109it [00:02, 40.18it/s]











114it [00:02, 40.41it/s]











119it [00:02, 39.56it/s]











123it [00:03, 39.28it/s]











127it [00:03, 39.05it/s]











131it [00:03, 39.00it/s]











135it [00:03, 38.89it/s]













Validation Loss Epoch: 0.5351313191094422
Validation Accuracy Epoch: 73.33333333333333
instagram
Accuracy on test data = 73.33%


In [ ]:
#v2
load_accuracy_for_labelled('instagram')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
1it [00:00,  3.62it/s]

Validation Loss per 100 steps: 0.7415665984153748
Validation Accuracy per 100 steps: 59.375


26it [00:06,  3.78it/s]

Validation Loss Epoch: 0.5653367867836585
Validation Accuracy Epoch: 71.9753086419753
instagram
Accuracy on test data = 71.98%


In [ ]:
#v3
load_accuracy_for_labelled('instagram')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
1it [00:00,  3.72it/s]

Validation Loss per 100 steps: 0.5939064621925354
Validation Accuracy per 100 steps: 75.0


26it [00:06,  3.79it/s]

Validation Loss Epoch: 0.5621654895635751
Validation Accuracy Epoch: 76.29629629629629
instagram
Accuracy on test data = 76.30%


In [ ]:
#v4
load_accuracy_for_labelled('instagram')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
1it [00:00,  3.44it/s]

Validation Loss per 100 steps: 0.5980224013328552
Validation Accuracy per 100 steps: 78.125


26it [00:07,  3.70it/s]

Validation Loss Epoch: 0.5584247754170344
Validation Accuracy Epoch: 76.29629629629629
instagram
Accuracy on test data = 76.30%


In [ ]:
load_accuracy_for_labelled('tiktok')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
1it [00:00,  3.59it/s]

Validation Loss per 100 steps: 0.5998286008834839
Validation Accuracy per 100 steps: 75.0


21it [00:05,  3.78it/s]

Validation Loss Epoch: 0.4924039109831765
Validation Accuracy Epoch: 78.03738317757009
tiktok
Accuracy on test data = 78.04%


In [ ]:
#v2 
load_accuracy_for_labelled('tiktok')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
1it [00:00,  3.70it/s]

Validation Loss per 100 steps: 0.2982826232910156
Validation Accuracy per 100 steps: 90.625


21it [00:05,  3.89it/s]

Validation Loss Epoch: 0.5042436250618526
Validation Accuracy Epoch: 78.03738317757009
tiktok
Accuracy on test data = 78.04%


In [ ]:
#v3
load_accuracy_for_labelled('tiktok')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
1it [00:00,  3.59it/s]

Validation Loss per 100 steps: 0.5874050855636597
Validation Accuracy per 100 steps: 78.125


21it [00:05,  3.78it/s]

Validation Loss Epoch: 0.48198395257904414
Validation Accuracy Epoch: 77.57009345794393
tiktok
Accuracy on test data = 77.57%


In [ ]:
#v4
load_accuracy_for_labelled('tiktok')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
1it [00:00,  3.52it/s]

Validation Loss per 100 steps: 0.4575605094432831
Validation Accuracy per 100 steps: 81.25


21it [00:05,  3.70it/s]

Validation Loss Epoch: 0.47484955688317615
Validation Accuracy Epoch: 77.57009345794393
tiktok
Accuracy on test data = 77.57%


In [ ]:
foutput_model_file = 'pytorch_roberta_sentiment.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')

All files saved
This tutorial is completed
